In [1]:
from sympy import symbols, Eq, Matrix
from symsolver.symsolver import SymSolver

In [2]:
s = [1, 2,
     "R", "I",
     "{C_1}", "{C_2}",
     "{A_1}", "{A_2}"]
idx_s = [1, 2,
         "R", "I",
         "C1", "C2",
         "A1", "A2"]

_P = symbols(
    " ".join([f"P_{n}" if n else "P" for n in s]))
_Pd = symbols(
    " ".join([f"\dot{{P}}_{n}" if n else "\dot{P}" for n in s]))
def P(s):
    idx = idx_s.index(s)
    return _P[idx]
def Pd(s):
    idx = idx_s.index(s)
    return _Pd[idx]

_Q = symbols(
    " ".join([f"Q_{n}" if n else "Q" for n in s]))
_Qd = symbols(
    " ".join([f"\dot{{Q}}_{n}" if n else "\dot{Q}" for n in s]))
def Q(s):
    idx = idx_s.index(s)
    return _Q[idx]
def Qd(s):
    idx = idx_s.index(s)
    return _Qd[idx]
R = symbols("R")
I = symbols("I")
_A = symbols("A_1 A_2")
def A(i):
    return _A[i-1]
_C = symbols("C_1 C_2")
def C(i):
    return _C[i-1]

In [3]:
basic_elements = [
                  Eq(C(1)*Pd("C1"), Q("C1")),
                  Eq(C(2)*Pd("C2"), Q("C2")),
                  Eq(I*Qd("I"), P("I")),
                  Eq(R*Q("R"), P("R"))
                  ]

In [4]:
r = A(1)/A(2)
transformers = [
                Eq(P("A2"), r*P("A1")),
                Eq(Q("A2"), -1/r*Q("A1"))
                ]

In [5]:
loop_equations = [
                  Eq(P(1), P("C1")),
                  Eq(P("C1"), P("R") +  P("I") +  P("A1")),
                  Eq(P("A2"), P("C2")),
                  Eq(P("C2"), P(2))
                  ]

In [6]:
node_equations = [
                  Eq(Q(1), Q("C1") + Q("R")),
                  Eq(Q("R"), Q("I")),
                  Eq(Q("I"), Q("A1")),
                  Eq(Q(2), Q("A2") + Q("C2"))
                  ]

In [7]:
x_1 = P("C1")
x_2 = Q("I")
x_3 = P("C2")
x = Matrix([
            [x_1],
            [x_2],
            [x_3]
            ])

x_1d = Pd("C1")
x_2d = Qd("I")
x_3d = Pd("C2")
xd = Matrix([
            [x_1d],
            [x_2d],
            [x_3d]
            ])

In [8]:
relations = basic_elements \
  + transformers \
  + loop_equations \
  + node_equations

state_sym = set.union(x.free_symbols, {Q(1), Q(2)})
const_sym = set(
    list(_C) + \
    list(_A) + \
    list((R, I)))
ok_sym = set.union(state_sym, const_sym)

In [9]:
solver = SymSolver(relations=relations, wanted_symbols=ok_sym)

In [10]:
solver.solve(x_1d)

Solving expression \dot{P}_{C_1}
Got child expression Q_{C_1}/C_1
Got child expression \dot{P}_{C_1}
Got child expression \dot{P}_{C_1}
Expression was a duplicate
Got child expression (Q_1 - Q_R)/C_1
Got child expression \dot{P}_{C_1}*(Q_1 - Q_R)/Q_{C_1}
Got child expression (-P_R/R + Q_1)/C_1
Got child expression Q_{C_1}/C_1
Expression was a duplicate
Got child expression Q_{C_1}/C_1
Expression was a duplicate
Got child expression (Q_1 - Q_I)/C_1
Got solution (Q_1 - Q_I)/C_1


(Q_1 - Q_I)/C_1

In [11]:
solver.solve(x_2d)

Solving expression \dot{Q}_I
Got child expression P_I/I
Got child expression \dot{Q}_I
Got child expression \dot{Q}_I
Expression was a duplicate
Got child expression (-P_R - P_{A_1} + P_{C_1})/I
Got child expression \dot{Q}_I*(-P_R - P_{A_1} + P_{C_1})/P_I
Got child expression (-P_{A_1} + P_{C_1} - Q_R*R)/I
Got child expression (-P_R + P_{C_1} - A_2*P_{A_2}/A_1)/I
Got child expression (P_1 - P_R - P_{A_1})/I
Got child expression P_I/I
Expression was a duplicate
Got child expression P_I/I
Expression was a duplicate
Got child expression P_I/I
Expression was a duplicate
Got child expression \dot{Q}_I*(-P_R + P_{C_1} - A_2*P_{A_2}/A_1)/P_I
Got child expression (P_{C_1} - Q_R*R - A_2*P_{A_2}/A_1)/I
Got child expression (-P_R - P_{A_1} + P_{C_1})/I
Expression was a duplicate
Got child expression (-P_R - P_{A_1} + P_{C_1})/I
Expression was a duplicate
Got child expression (-P_R - P_{A_1} + P_{C_1})/I
Expression was a duplicate
Got child expression (-P_R + P_{A_2}*Q_{A_2}/Q_{A_1} + P_{C_1})/I


(P_{C_1} - Q_I*R - A_2*P_{C_2}/A_1)/I

In [12]:
solver.solve(x_3d)

Solving expression \dot{P}_{C_2}
Got child expression Q_{C_2}/C_2
Got child expression \dot{P}_{C_2}
Got child expression \dot{P}_{C_2}
Expression was a duplicate
Got child expression (Q_2 - Q_{A_2})/C_2
Got child expression \dot{P}_{C_2}*(Q_2 - Q_{A_2})/Q_{C_2}
Got child expression (Q_2 + A_2*Q_{A_1}/A_1)/C_2
Got child expression Q_{C_2}/C_2
Expression was a duplicate
Got child expression Q_{C_2}/C_2
Expression was a duplicate
Got child expression \dot{P}_{C_2}*(Q_2 + A_2*Q_{A_1}/A_1)/Q_{C_2}
Got child expression (P_{A_1}*Q_{A_1}/P_{A_2} + Q_2)/C_2
Got child expression (P_{A_1}*Q_{A_1}/P_{A_2} + Q_2)/C_2
Expression was a duplicate
Got child expression (Q_2 - Q_{A_2})/C_2
Expression was a duplicate
Got child expression (Q_2 - Q_{A_2})/C_2
Expression was a duplicate
Got child expression (Q_2 - Q_{A_2})/C_2
Expression was a duplicate
Got child expression (Q_2 + A_2*Q_I/A_1)/C_2
Got solution (Q_2 + A_2*Q_I/A_1)/C_2


(Q_2 + A_2*Q_I/A_1)/C_2

In [ ]:
7